In [1]:
import tensorflow as tf
import numpy as np
import os

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping, TensorBoard

physical_devices = tf.config.experimental.list_physical_devices('GPU')

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("this happened")

this happened


In [2]:
import pickle

folder = 'pure_amd_pickled'

pickle_in = open(folder + '/' + 'ODIR_AMD_w_others_pred.pickle','rb')
ODIR_AMD_w_others = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'ODIR_Pure_AMD_pred.pickle','rb')
ODIR_Pure_AMD = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'RFMID_AMD_w_others_pred.pickle','rb')
RFMID_AMD_w_others = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'RFMID_Pure_AMD_pred.pickle','rb')
RFMID_Pure_AMD = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'RFMID_normal_pred.pickle','rb')
RFMID_normal = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'stare_AMD_w_others_pred.pickle','rb')
stare_AMD_w_others = pickle.load(pickle_in)

pickle_in = open(folder + '/' + 'stare_normal_pred.pickle','rb')
stare_normal = pickle.load(pickle_in)

In [3]:
print(len(ODIR_AMD_w_others))
print(len(ODIR_Pure_AMD))
print(len(RFMID_AMD_w_others))
print(len(RFMID_Pure_AMD))
print(len(stare_AMD_w_others))
print(len(RFMID_normal[:480]))
print(len(stare_normal))

print('amd = ', len(stare_AMD_w_others) + len(RFMID_Pure_AMD) + len(RFMID_AMD_w_others) + len(ODIR_Pure_AMD) + len(ODIR_AMD_w_others))

76
258
132
36
72
480
80
amd =  574


In [4]:
segmented_amd_mixed = np.concatenate((ODIR_AMD_w_others, ODIR_Pure_AMD, RFMID_AMD_w_others, RFMID_Pure_AMD, stare_AMD_w_others), axis=0)
segmented_pure_amd = np.concatenate((ODIR_Pure_AMD, RFMID_Pure_AMD), axis=0)
segmented_normal = np.concatenate((RFMID_normal[:480], stare_normal), axis=0)

In [5]:
y_amd = []
y_normal = []

for i in range(len(segmented_amd_mixed)):
    y_amd.append(1)

for i in range(len(segmented_normal)):
    y_normal.append(0)

label_amd = np.array(y_amd)
label_normal = np.array(y_normal)

In [6]:
x = np.concatenate((segmented_amd_mixed, segmented_normal), axis=0)
y = np.concatenate((label_amd, label_normal), axis=0)

In [7]:
#x = np.array(x/255.0)

In [8]:
from sklearn.model_selection import train_test_split

x_train, alpha, y_train, omega = train_test_split(x, y, test_size=0.2, random_state=1)
x_eval, x_test, y_eval, y_test = train_test_split(alpha, omega, test_size=0.5, random_state=1)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(512, 512, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1016064)           0         
                                                                 
 dense (Dense)               (None, 128)               1

In [10]:
# Define the batch size and number of epochs for training
batch_size = 4
epochs = 100
model_filename = 'method_v1.h5'
callback_checkpoint =[EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=False),
ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, min_lr=1e-7, verbose=1),
TensorBoard()]

In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_eval.shape)
print(y_eval.shape)

(907, 512, 512, 1)
(907,)
(113, 512, 512, 1)
(113,)


In [12]:
with tf.device("GPU"):
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_eval, y_eval),
        callbacks=[callback_checkpoint])

Epoch 1/100
227/227 [==============================] - ETA: 0s - loss: 0.5475 - accuracy: 0.7409
Epoch 1: val_loss improved from inf to 0.52884, saving model to method_v1.h5
227/227 [==============================] - 27s 98ms/step - loss: 0.5475 - accuracy: 0.7409 - val_loss: 0.5288 - val_accuracy: 0.8142 - lr: 0.0010
Epoch 2/100
227/227 [==============================] - ETA: 0s - loss: 0.4821 - accuracy: 0.7982
Epoch 2: val_loss did not improve from 0.52884
227/227 [==============================] - 11s 50ms/step - loss: 0.4821 - accuracy: 0.7982 - val_loss: 0.6535 - val_accuracy: 0.7965 - lr: 0.0010
Epoch 3/100
227/227 [==============================] - ETA: 0s - loss: 0.4048 - accuracy: 0.8490
Epoch 3: val_loss did not improve from 0.52884
227/227 [==============================] - 11s 49ms/step - loss: 0.4048 - accuracy: 0.8490 - val_loss: 0.5663 - val_accuracy: 0.8053 - lr: 0.0010
Epoch 4/100
227/227 [==============================] - ETA: 0s - loss: 0.2885 - accuracy: 0.8875
Epo

In [13]:
model.evaluate(x_test, y_test, batch_size=4)

29/29 [==============================] - 0s 15ms/step - loss: 0.8240 - accuracy: 0.8684


[0.823965847492218, 0.8684210777282715]

In [14]:
model.load_weights(model_filename)
with tf.device("GPU"): 
    result = model.predict(x_test)

print('Predictions: ', len(result))

4/4 [==============================] - 2s 235ms/step
Predictions:  114


In [15]:
import pandas as pd

data = { 'Actual': y_test, 'Predicted': result }
preds = pd.DataFrame(data)

preds

ValueError: Per-column arrays must each be 1-dimensional

In [16]:
result

array([[2.2109169e-01],
       [5.6711721e-01],
       [3.0537513e-01],
       [2.2109175e-01],
       [2.2109175e-01],
       [5.1980180e-01],
       [6.6258299e-01],
       [7.1378791e-01],
       [3.8081431e-01],
       [7.5118470e-01],
       [2.2109169e-01],
       [7.8549874e-01],
       [7.2677970e-01],
       [2.2109175e-01],
       [9.0685475e-04],
       [4.9718860e-01],
       [2.9597819e-01],
       [2.2109175e-01],
       [2.2109175e-01],
       [2.2109175e-01],
       [3.9299610e-01],
       [4.7605953e-01],
       [2.2109175e-01],
       [2.2109175e-01],
       [9.8928553e-01],
       [5.8181775e-01],
       [4.8580456e-01],
       [9.7520918e-01],
       [9.9845421e-01],
       [4.3961406e-01],
       [9.8902273e-01],
       [9.9994576e-01],
       [6.8029606e-01],
       [5.2041441e-01],
       [9.9885821e-01],
       [2.2109175e-01],
       [2.2109175e-01],
       [6.1062068e-01],
       [2.2109175e-01],
       [7.0001203e-01],
       [2.2109175e-01],
       [9.068023